In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [910 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-08-22 07:37:35--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  6.02MB/s    in 0.2s    

2022-08-22 07:37:36 (6.02 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
#spark session creation
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [15]:
from pyspark import SparkFiles
url = "https://unixbears-bucket.s3.amazonaws.com/amazon_reviews_us_Digital_Video_Games_v1_00.tsv"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   21269168| RSH1OZ87OYK92|B013PURRZW|     603406193|Madden NFL 16 - X...|Digital_Video_Games|          2|            2|          3|   N|                N|A slight improvem...|I keep buying mad...|2015-08-31 00:00:00|
|         US|     133437|R1WFOQ3N9BO65I|B00F4CEHNK|     341969535| Xbox Live

In [8]:
#getting db password to load into dataframes
from getpass import getpass
password = getpass("enter pwd plz")

enter pwd plz··········


In [9]:
vine_df = spark.read.format("jdbc").option("url", "jdbc:postgresql://mod16-challenge.c4yohomp20p9.us-east-1.rds.amazonaws.com/vine_game_reviews") \
  .option("driver", "org.postgresql.Driver").option("dbtable", "vine_table") \
  .option("user", "postgres") \
  .option("password", password).load()

In [10]:
review_id_df = spark.read.format("jdbc").option("url", "jdbc:postgresql://mod16-challenge.c4yohomp20p9.us-east-1.rds.amazonaws.com/vine_game_reviews") \
  .option("driver", "org.postgresql.Driver").option("dbtable", "review_id_table") \
  .option("user", "postgres") \
  .option("password", password).load()

In [11]:
products_df = spark.read.format("jdbc").option("url", "jdbc:postgresql://mod16-challenge.c4yohomp20p9.us-east-1.rds.amazonaws.com/vine_game_reviews") \
  .option("driver", "org.postgresql.Driver").option("dbtable", "products_table") \
  .option("user", "postgres") \
  .option("password", password).load()

In [12]:
customers_df = spark.read.format("jdbc").option("url", "jdbc:postgresql://mod16-challenge.c4yohomp20p9.us-east-1.rds.amazonaws.com/vine_game_reviews") \
  .option("driver", "org.postgresql.Driver").option("dbtable", "customers_table") \
  .option("user", "postgres") \
  .option("password", password).load()

In [13]:
vine_df.show(5)
review_id_df.show(5)
products_df.show(5)
customers_df.show(5)


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R142GG01FCHFZ2|          1|            2|          2|   N|                Y|
|R1WU37CVMMD6J3|          1|            0|          3|   N|                N|
|R11C9OF30Q1NY5|          3|            0|          0|   N|                Y|
| RCP2HCB08FP71|          5|            0|          0|   N|                N|
|R1Q9XARTW5C6HT|          2|            0|          0|   N|                Y|
+--------------+-----------+-------------+-----------+----+-----------------+
only showing top 5 rows

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RSH1OZ87OYK92|   21269168|B013PURRZW|     603406193| 2015-08-31|
|R1WF

1) Filter the data and create a new DataFrame or table to retrieve all the rows where the `total_votes` count is equal to or greater than 20 to pick reviews that are more likely to be helpful and to avoid having division by zero errors later on.

In [17]:
cleanedDF = df.where(df.total_votes >= 20)
cleanedDF.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...|2015-08-29 00:00:00|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket Lea

2) Filter the new DataFrame or table created in Step 1 and create a new DataFrame or table to retrieve all the rows where the number of `helpful_votes` divided by `total_votes` is equal to or greater than 50%.

  - If you use the SQL option below, you’ll need to cast your columns as floats using WHERE CAST(helpful_votes AS FLOAT)/CAST(total_votes AS FLOAT) >=0.5.



In [18]:
vineDF = cleanedDF.where((cleanedDF.helpful_votes / cleanedDF.total_votes) >= 0.5)
vineDF.show(5)

+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|   product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+-------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   12569493| RV4UH0OWW97K4|B0149HT55K|     947161997|Megaman Legacy Co...|Digital_Video_Games|          5|           19|         22|   N|                N|Mega Man Legacy C...|What is Mega Man?...|2015-08-29 00:00:00|
|         US|   40702997| RG6GT6G9WNYT1|B013XJ2M8M|     628348017|Rocket Lea

Filter the DataFrame or table created in Step 2, and create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), `vine == 'Y'`.

In [47]:
#paidVineDF is empty
reviewCount = vineDF.count()
fiveStarReviews = vineDF.where(vineDF.star_rating == 5).count()
fiveStarReviews

631

Percentage of 5-star reviews by paid: 0%
Percentage of 5-star reviews by unpaid: 100%

